### Read in predictions from all models

In [2]:
import pandas as pd
import numpy as np

In [3]:
models = ["BERT","fasttext", "logreg", "NB", "RF", "SVM","VADER"]
meta_model_train = pd.DataFrame()
meta_model_test = pd.DataFrame()
meta_model_fold_1 = pd.DataFrame()
meta_model_fold_2 = pd.DataFrame()
meta_model_fold_3 = pd.DataFrame()
meta_model_fold_4 = pd.DataFrame()
meta_model_fold_5 = pd.DataFrame()

for model in models:
    fold1pred = pd.read_csv(f'fold_predictions/{model}/{model}_fold1.csv')
    meta_model_fold_1 = pd.concat([meta_model_fold_1,fold1pred], axis=1)
    
    fold2pred = pd.read_csv(f'fold_predictions/{model}/{model}_fold2.csv')
    meta_model_fold_2 = pd.concat([meta_model_fold_2,fold2pred], axis=1)

    fold3pred = pd.read_csv(f'fold_predictions/{model}/{model}_fold3.csv')
    meta_model_fold_3 = pd.concat([meta_model_fold_3,fold3pred], axis=1)

    fold4pred = pd.read_csv(f'fold_predictions/{model}/{model}_fold4.csv')
    meta_model_fold_4 = pd.concat([meta_model_fold_4,fold4pred], axis=1)

    fold5pred = pd.read_csv(f'fold_predictions/{model}/{model}_fold5.csv')    
    meta_model_fold_5 = pd.concat([meta_model_fold_5,fold5pred], axis=1)

    all_pred_train = pd.concat([fold1pred,fold2pred,fold3pred,fold4pred,fold5pred],axis = 0)
    meta_model_train = pd.concat([meta_model_train,all_pred_train], axis=1)
    
    testpred = pd.read_csv(f'fold_predictions/{model}/{model}_test.csv')
    meta_model_test = pd.concat([meta_model_test,testpred], axis=1)

    
data = [meta_model_fold_1,meta_model_fold_2,meta_model_fold_3,meta_model_fold_4,meta_model_fold_5,meta_model_train]
meta_model_train.head()


,bert_prob_pos,bert_prob_neg,fasttext_prob_pos,fasttext_prob_neg,logreg_prob_pos,logreg_prob_neg,NB_prob_pos,NB_prob_neg,RF_prob_pos,RF_prob_neg,SVM_prob_pos,SVM_prob_neg,VADER_prob_pos,VADER_prob_neg,label
0,0.967741,0.005076,0.563547,0.214631,0.939938,0.004414,0.833302,0.006866,0.950000,0.000000,0.761402,0.014256,0.592,0.000,1.0
1,0.976611,0.005581,0.573885,0.212415,0.998430,0.000544,0.823375,0.052906,0.940909,0.000000,0.884794,0.020397,1.000,0.000,1.0
2,0.142880,0.464640,0.368451,0.287994,0.479545,0.127199,0.548471,0.098711,0.711548,0.251786,0.675172,0.081166,0.145,0.150,0.0
3,0.199171,0.040443,0.311945,0.353513,0.172376,0.229518,0.043481,0.038845,0.319444,0.416667,0.600380,0.136811,0.154,0.203,1.0
4,0.972743,0.005263,0.431019,0.244249,0.740021,0.065695,0.867578,0.029769,0.825000,0.000000,0.852314,0.023726,0.765,0.000,1.0


# Modelling

In [4]:
import pandas as pd
pd.set_option('display.max_rows', 100)
import numpy as np
import utils

from sklearn.feature_extraction.text import CountVectorizer 
import re
from tqdm import tqdm

from sklearn.model_selection import ParameterGrid

from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [5]:
# logistic regression
logreg_params = {
    "C": [0.1, 0.5, 1.0, 1.5, 5],
    "solver": ["lbfgs", "newton-cg"],
    "penalty": ["l2", "none"],
    "class_weight": ["balanced", None] 
}
logreg_paramgrid = list(ParameterGrid(logreg_params))

In [6]:
# naive bayes
nb_params = {
    "alpha": [0, 0.001, 0.01, 0.1, 0.25, 0.5, 1]
}
nb_paramgrid = list(ParameterGrid(nb_params))

In [7]:
# svm
svm_params = {
    "C": [0.1, 0.5, 1.0, 1.5, 5],
    "kernel": ["poly", "rbf", "sigmoid"],
    "gamma": ["scale", "auto"],
    "class_weight": ["balanced", None] 
}

svm_paramgrid = list(ParameterGrid(svm_params))

In [8]:
rf_params = [
    {
        "criterion": ["gini", "entropy"],
        "min_samples_split": [2, 5, 10],
        "class_weight": ["balanced", "balanced_subsample", None],
        "max_features": ["auto","sqrt"],
        "min_samples_leaf": [1, 2, 4]
    }
]

rf_paramgrid = list(ParameterGrid(rf_params))

In [9]:
dummy_params = { "strategy": ["prior"] }
dummy_paramgrid = list(ParameterGrid(dummy_params))

# Grid Search 

### Logistic Regression/NB/SVM/DummyClassifier

In [10]:
def models_grid_search(model_name, model_fn, model_paramgrid,data, test) : 
    ind = 0 
    gridsearch_results = []      
    
    # train models
    for model_param in model_paramgrid:

        # 5 fold cross val
        val_accuracy = []
        val_f1_weighted = []
        val_f1_neg = []
        val_f1_zero = []
        val_f1_pos = []
    
        test_accuracy = []
        test_f1_weighted = []
        test_f1_neg = []
        test_f1_zero = []
        test_f1_pos = []
        
        for i in range(5):
            print(f"fold {i}")
            train_set = pd.DataFrame()
            for x in range(5):
                if i != x :
                    train_set = pd.concat([train_set,data[x]],axis=0)
            val_set = data[i]
            test_set = test
            trainval_set = data[5]

            train_label = train_set.label
            val_label = val_set.label
            test_label = test_set.label
            trainval_label = trainval_set.label

            # train on train model test on val
            model = model_fn(**model_param)
            model.fit(train_set.iloc[:,:-1], train_label)
            val_pred = model.predict(val_set.iloc[:,:-1])
            # scoring
            val_metrics = classification_report(val_label, val_pred, output_dict=True)
            val_accuracy.append(val_metrics["accuracy"])
            val_f1_weighted.append(val_metrics["weighted avg"]["f1-score"])
            val_f1_neg.append(val_metrics["-1.0"]["f1-score"])
            val_f1_zero.append(val_metrics["0.0"]["f1-score"])
            val_f1_pos.append(val_metrics["1.0"]["f1-score"])

            # train on train_val model test on test
            model = model_fn(**model_param)
            model.fit(trainval_set.iloc[:,:-1], trainval_label)
            test_pred = model.predict(test_set.iloc[:,:-1])

            # scoring
            test_metrics = classification_report(test_label, test_pred, output_dict=True)
            test_accuracy.append(test_metrics["accuracy"])
            test_f1_weighted.append(test_metrics["weighted avg"]["f1-score"])
            test_f1_neg.append(test_metrics["-1.0"]["f1-score"])
            test_f1_zero.append(test_metrics["0.0"]["f1-score"])
            test_f1_pos.append(test_metrics["1.0"]["f1-score"])

        results = { "model": model_name }
        results.update(model_param)
        results.update({"val_f1_weighted": np.mean(val_f1_weighted), "val_f1_neg": np.mean(val_f1_neg), 
                        "val_f1_zero": np.mean(val_f1_zero), "val_f1_pos": np.mean(val_f1_pos),
                        "val_accuracy": np.mean(val_accuracy)})
        results.update({"test_f1_weighted": np.mean(test_f1_weighted), "test_f1_neg": np.mean(test_f1_neg), 
                        "test_f1_zero": np.mean(test_f1_zero), "test_f1_pos": np.mean(test_f1_pos),
                        "test_accuracy": np.mean(test_accuracy)})
        print(results)
        gridsearch_results.append(results)
        ind += 1
    return gridsearch_results
        

In [11]:
model_name = "logreg"
model_fn = LogisticRegression
model_paramgrid = logreg_paramgrid
final_logreg_results = pd.DataFrame.from_records(models_grid_search(model_name,model_fn,model_paramgrid, data, meta_model_test))

model_name = "nb"
model_fn = MultinomialNB
model_paramgrid = nb_paramgrid
final_nb_results = pd.DataFrame.from_records(models_grid_search(model_name,model_fn,model_paramgrid, data, meta_model_test))

model_name = "svm"
model_fn = SVC
model_paramgrid = svm_paramgrid
final_svm_results = pd.DataFrame.from_records(models_grid_search(model_name,model_fn,model_paramgrid, data, meta_model_test))

model_name = "dummy"
model_fn = DummyClassifier
model_paramgrid = dummy_paramgrid
final_dummy_results = pd.DataFrame.from_records(models_grid_search(model_name,model_fn,model_paramgrid, data, meta_model_test))

151942, 'val_f1_neg': 0.6820185793818586, 'val_f1_zero': 0.7560752762056623, 'val_f1_pos': 0.8512163969307085, 'val_accuracy': 0.7951935148295707, 'test_f1_weighted': 0.780176896049158, 'test_f1_neg': 0.6564102564102564, 'test_f1_zero': 0.7174348697394789, 'test_f1_pos': 0.838785046728972, 'test_accuracy': 0.7767741935483871}
fold 0
fold 1
fold 2
fold 3
fold 4
{'model': 'svm', 'C': 0.5, 'class_weight': None, 'gamma': 'scale', 'kernel': 'sigmoid', 'val_f1_weighted': 0.3397153938783571, 'val_f1_neg': 0.305178336778147, 'val_f1_zero': 0.2510658981019434, 'val_f1_pos': 0.4138239711092389, 'val_accuracy': 0.3489274141529121, 'test_f1_weighted': 0.4014242288371584, 'test_f1_neg': 0.2885085574572127, 'test_f1_zero': 0.30985915492957744, 'test_f1_pos': 0.4714119019836639, 'test_accuracy': 0.3935483870967742}
fold 0
fold 1
fold 2
fold 3
fold 4
{'model': 'svm', 'C': 0.5, 'class_weight': None, 'gamma': 'auto', 'kernel': 'poly', 'val_f1_weighted': 0.6720126335331523, 'val_f1_neg': 0.30406421776768

In [12]:
final_logreg_results = final_logreg_results.sort_values(by=["val_f1_weighted", "test_f1_weighted"], ascending=False)
final_nb_results = final_nb_results.sort_values(by=["val_f1_weighted", "test_f1_weighted"], ascending=False)
final_svm_results = final_svm_results.sort_values(by=["val_f1_weighted", "test_f1_weighted"], ascending=False)
final_dummy_results = final_dummy_results.sort_values(by=["val_f1_weighted", "test_f1_weighted"], ascending=False)

In [13]:
final_logreg_results.iloc[0,:]

model                 logreg
C                        0.1
class_weight        balanced
penalty                   l2
solver                 lbfgs
val_f1_weighted          0.8
val_f1_neg          0.697722
val_f1_zero         0.763165
val_f1_pos          0.852187
val_accuracy        0.798951
test_f1_weighted    0.781973
test_f1_neg         0.688073
test_f1_zero        0.714286
test_f1_pos         0.836105
test_accuracy       0.776774
Name: 0, dtype: object

In [14]:
final_nb_results.iloc[0,:]

model                     nb
alpha                      0
val_f1_weighted     0.694378
val_f1_neg          0.653601
val_f1_zero         0.575207
val_f1_pos          0.793207
val_accuracy         0.70723
test_f1_weighted    0.749485
test_f1_neg         0.659091
test_f1_zero        0.607059
test_f1_pos         0.838778
test_accuracy       0.754839
Name: 0, dtype: object

In [15]:
final_svm_results.iloc[0,:]

model                    svm
C                        0.5
class_weight        balanced
gamma                  scale
kernel                   rbf
val_f1_weighted     0.798112
val_f1_neg          0.703557
val_f1_zero         0.758925
val_f1_pos          0.849474
val_accuracy         0.79656
test_f1_weighted    0.783189
test_f1_neg         0.707965
test_f1_zero        0.714876
test_f1_pos         0.833333
test_accuracy       0.778065
Name: 13, dtype: object

In [16]:
final_dummy_results.iloc[0,:]

model                  dummy
strategy               prior
val_f1_weighted     0.330713
val_f1_neg                 0
val_f1_zero                0
val_f1_pos           0.66364
val_accuracy        0.497176
test_f1_weighted    0.431247
test_f1_neg                0
test_f1_zero               0
test_f1_pos         0.737785
test_accuracy       0.584516
Name: 0, dtype: object

### Train on all data & save model

In [41]:
alldata = pd.concat([meta_model_train,meta_model_test],axis=0)

In [40]:
# save model trained on all data
import pickle
best_param = {"C": final_logreg_results.iloc[0,:].to_dict()["C"],
"class_weight" : final_logreg_results.iloc[0,:].to_dict()["class_weight"] , "penalty" : final_logreg_results.iloc[0,:].to_dict()["penalty"], "solver" : final_logreg_results.iloc[0,:].to_dict()["solver"]}
final_model = LogisticRegression(**best_param)
final_model.fit(alldata.iloc[:,:-1], alldata.label)

model_pkl_filename = "saved_models/model_meta.pkl"
with open(model_pkl_filename, 'wb') as file:
    pickle.dump(final_model, file)

#### Function

#### Save Model